# Libraries

In [1]:
import pandas as pd
import torch
import os

from config import config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, AutoModelForCausalLM

In [2]:
print('GPU: ', torch.cuda.is_available())
x = torch.cuda.current_device()
print('GPU Device: ', torch.cuda.get_device_name(x))
print('GPU Memory: ', torch.cuda.max_memory_allocated())
print('Number of GPUs: ', torch.cuda.device_count())

GPU:  True
GPU Device:  NVIDIA A100 80GB PCIe
GPU Memory:  0
Number of GPUs:  2


# Testing

In [3]:
# project_path = config['project_path']
# model_path = os.path.join(project_path, 'models', 'Meta-Llama-3.2-3B-Instruct')

# pipe = pipeline(
#     "text-generation",
#     model=model_path,
#     torch_dtype=torch.bfloat16,
#     device_map="auto",
# )
# messages = [
#     {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
#     {"role": "user", "content": "Who are you?"},
# ]
# outputs = pipe(
#     messages,
#     max_new_tokens=256,
# )
# print(outputs[0]["generated_text"][-1])

In [4]:
# tokenizer = AutoTokenizer.from_pretrained(model_path)
# model = AutoModelForCausalLM.from_pretrained(model_path)

# Data Preparation

In [5]:
# data = pd.read_csv('dataset/symbipredict_2022.csv')
# data.head()

In [6]:
# label_encoder = LabelEncoder()
# data['prognosis'] = label_encoder.fit_transform(data['prognosis'])

In [7]:
# data.head()

In [8]:
# train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
# train_data, val_data = train_test_split(train_data, test_size=0.1, random_state=42)

# Model Setup

In [9]:
# # Load model directly
# model_path = 'models/Llama-3.1-Nemotron-70B-Instruct-HF/'

# model = AutoModelForSequenceClassification.from_pretrained(model_path)
# tokenizer = AutoTokenizer.from_pretrained(model_path)

# OpenBioLLM-70B 

## Clear GPU Memory

In [10]:
torch.cuda.empty_cache()

### Sample Test

In [11]:
model_id = "models/OpenBioLLM-Llama3-70B"

# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)

In [14]:
!PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [15]:
pipeline_gen = pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device="cuda",
)

messages = [
    {"role": "system", "content": "You are an expert and experienced from the healthcare and biomedical domain with extensive medical knowledge and practical experience. Your name is OpenBioLLM, and you were developed by Saama AI Labs. who's willing to help answer the user's query with explanation. In your explanation, leverage your deep medical expertise such as relevant anatomical structures, physiological processes, diagnostic criteria, treatment guidelines, or other pertinent medical concepts. Use precise medical terminology while still aiming to make the explanation clear and accessible to a general audience."},
    {"role": "user", "content": "How can i split a 3mg or 4mg waefin pill so i can get a 2.5mg pill?"},
]

prompt = pipeline_gen.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
)

terminators = [
    pipeline_gen.tokenizer.eos_token_id,
    pipeline_gen.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = pipeline_gen(
    prompt,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
)

Loading checkpoint shards:   0%|          | 0/30 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.96 GiB. GPU 0 has a total capacity of 79.14 GiB of which 112.75 MiB is free. Including non-PyTorch memory, this process has 79.02 GiB memory in use. Of the allocated memory 78.61 GiB is allocated by PyTorch, and 488.00 KiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print(outputs[0]["generated_text"][len(prompt):])

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [ ]:
print(torch.__version__)

In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name(torch.cuda.current_device()))

In [ ]:
os.environ['TORCH_USE_CUDA_DSA'] = '1'